In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/') 
sys.path.append('../')
import RNA 
from explorers.dqn_explorer import DQN_Explorer 
from explorers.bo_explorer import BO_Explorer
from evaluators.Evaluator import Evaluator

LANDSCAPE_TYPES ={"RNA": [2,3],"TF": [1]}

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer()
dqn_explorer.debug = True
evaluator_dqn=Evaluator(dqn_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=1)

# evaluate BO Explorer 
bo_explorer = BO_Explorer()
bo_explorer.debug = True
evaluator_bo=Evaluator(bo_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=1)

Using TensorFlow backend.


Q_Network(
  (linear1): Linear(in_features=320, out_features=160, bias=True)
  (bn1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=160, out_features=40, bias=True)
  (bn2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=40, out_features=1, bias=True)
)
loading landscapes RNA: [2, 3], TF:[1]
B1L100RNA1 loaded
B1L14RNA2 loaded
1 TF landscapes loaded.
loading complete
Running on B1L100RNA1
start seq startRNAL100_0


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)
